In [1]:
from itertools import chain
import pycrfsuite
import sklearn
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import LeaveOneOut

In [2]:
# コーパス読み込み
import codecs
class CorpusReader(object):
    
    def __init__(self, path):
        with codecs.open(path, encoding='utf-8') as f:
            sent = []
            sents = []
            for line in f:
                if line == '\n':
                    sents.append(sent)
                    sent = []
                    continue
                morph_info = line.strip().split('\t')
                sent.append(morph_info) # 形態素の保存 
        train_num = int(len(sents) * 0.9) # 9割を学習に、1割をテストに
        self.__train_sents = sents[:train_num]
        self.__test_sents = sents[train_num:]
        self.__all_sents = sents
        
    def iob_sents(self, name):
        if name == 'train':
            return self.__train_sents
        elif name == 'test':
            return self.__test_sents
        elif name == 'all':
            return self.__all_sents
        else:
            return None
    

In [3]:
# 文字種取得
def is_hiragana(ch):
    return 0x3040 <= ord(ch) <= 0x309F 
    # ひらがな：True or False

def is_katakana(ch):
    return 0x30A0 <= ord(ch) <= 0x30FF
    # カタカタ：True or False

def get_character_type(ch): # 文字種を取得する
    if ch.isspace(): # 空白の場合
        return 'ZSPACE'
    elif ch.isdigit(): # 数字の場合
        return 'ZDIGIT'
    elif ch.islower(): # 小文字の場合
        return 'ZLLET'
    elif ch.isupper(): # 大文字の場合
        return 'ZULET'
    elif is_hiragana(ch): # ひらがなの場合
        return 'HIRAG'
    elif is_katakana(ch): # カタカナの場合
        return 'KATAK'
    else: # それ以外
        return 'OTHER'

def get_character_types(string): # 文字列の文字種を変換する
    character_types = map(get_character_type, string)
    character_types_str = '-'.join(sorted(set(character_types)))

    return character_types_str

In [4]:
# 品詞細分類の取得
def extract_pos_with_subtype(morph):
    idx = morph.index('*')
    return '-'.join(morph[1:idx])

In [5]:
# 単語を特徴量に変換する
def word2features(sent, i):
    word = sent[i][0]
    chtype = get_character_types(sent[i][0]) # 文字種取得
    postag = extract_pos_with_subtype(sent[i]) # 品詞分類取得
    
    # 該当単語の前後2文字の単語の特徴を用意
    features = [ 
        'bias',
        'word=' + word,
        'type=' + chtype,
        'postag=' + postag,
    ]
    
    if i >= 2: # 現在の単語の前に、2単語以上あるとき
        word2 = sent[i-2][0]
        chtype2 = get_character_types(sent[i-2][0])
        postag2 = extract_pos_with_subtype(sent[i-2])
        iobtag2 = sent[i-2][-1]
        features.extend([
            '-2:word=' + word2,
            '-2:type=' + chtype2,
            '-2:postag=' + postag2,
        ])
    else: # それ以外は、BOS
        features.append('BOS')

    if i >= 1: # 現在の単語の前に、1単語以上あるとき
        word1 = sent[i-1][0]
        chtype1 = get_character_types(sent[i-1][0])
        postag1 = extract_pos_with_subtype(sent[i-1])
        iobtag1 = sent[i-1][-1]
        features.extend([
            '-1:word=' + word1,
            '-1:type=' + chtype1,
            '-1:postag=' + postag1,
        ])
    else: # それ以外は、BOS
        features.append('BOS')

    if i < len(sent)-1: # 現在の単語の後ろに、1単語以上あるとき
        word1 = sent[i+1][0]
        chtype1 = get_character_types(sent[i+1][0])
        postag1 = extract_pos_with_subtype(sent[i+1])
        features.extend([
            '+1:word=' + word1,
            '+1:type=' + chtype1,
            '+1:postag=' + postag1,
        ])
    else: # それ以外は、EOS
        features.append('EOS')

    if i < len(sent)-2: # 現在の単語の後ろに、2単語以上あるとき
        word2 = sent[i+2][0]
        chtype2 = get_character_types(sent[i+2][0])
        postag2 = extract_pos_with_subtype(sent[i+2])
        features.extend([
            '+2:word=' + word2,
            '+2:type=' + chtype2,
            '+2:postag=' + postag2,
        ])
    else: # それ以外は、EOS
        features.append('EOS')

    return features    

def sent2features(sent): # 情報系列から特徴を取得
    # 単語ごとに特徴変換していく
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent): # 情報系列からラベル[B、I、O]を取得
    return [morph[-1] for morph in sent]

def sent2tokens(sent): # 情報系列から単語原文を取得
    return [morph[0] for morph in sent]


In [6]:
# ラベル評価
def bio_classification_report(y_true, y_pred):
    lb = LabelBinarizer()
    # 正解ラベルの二値化の保存 (1文字ずつのラベルを二値化に変換)
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    # 予測結果の二値化の保存 (1文字ずつのラベルを二値化に変換)
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))  

    tagset = set(lb.classes_) - {'O'} # O以外のタグセットの保存

    # B-NAME、I-NAME、B-THEME..等の順番に並び替える
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    # タグのクラスのid化？
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    # 正解ラベルと予想ラベルを引数として、評価表の作成をする
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )


In [7]:
# ラベルごとに単語を保存する
def label_report(text,label_data):
    wh_dic = {"WHERE":[], "WHEN":[], "WHO":[], "WHAT":[], "HOW":[], "WHY":[],"SERIF":[],"O":[]}
    
    set_label = ""
    result_word = ""
    wc = -1
    wc_l = []
    
    for i,word in enumerate(text):
        
        ld = label_data[i]
        
        # B,Iタグをとりあえず、無視する
        if "B-" in ld:
            ld = ld.replace("B-", "")
        elif "I-" in ld:
            ld = ld.replace("I-", "")
            
        # 最後の要素のとき
        if i == (len(text)-1):
            wh_dic[set_label].append([result_word,wc_l])
            break
        
        # ラベルごとに辞書へ保存する。
        if set_label != "" and set_label != ld: 
            # 辞書に保存            
            wh_dic[set_label].append([result_word,wc_l])
            
            # 初期化
            result_word = ""
            wc_l = []

        # 同一ラベルの情報を結合
        for wi in range(len(word)):
            wc += 1
            wc_l.append(wc)
        
        # 同一ラベルの情報を結合
        set_label = ld
        result_word += word   
            
    return wh_dic
    

In [8]:
"""
評価用
入力：正解データ辞書、予測データ辞書
出力： 5W1Hごとの失敗パターン辞書

===
完全： 正解データ
一部一致： 正解データ/予測データ
該当なし： 正解データ
誤予測： 予測データ

"""
def eval_report(true_wh_dic,pred_wh_dic):
    report_data = {}
    
    
    for k,v in true_wh_dic.items():
        report_v = {}
        result_type1 = []
        result_type2 = []
        result_type3 = []
        result_type4 = []
        
        check_pred = [0 for i in pred_wh_dic[k]]
        for true_text in v:    
            true_id = true_text[1] 

            if len(pred_wh_dic[k]) == 0: # 予想辞書にデータが存在しないとき
                result_type3.append(true_text[0])
                
            else: # 予想辞書にデータが存在するとき
                check_true = 0 # 正解データが予想辞書データとマッチングしたか確認
                for num, pred_text in enumerate(pred_wh_dic[k]):
                    pred_id = pred_text[1]                
                    match_n = list(set(true_id) & set(pred_id))
                    if len(match_n) > 0:
                        if len(true_id) == len(pred_id):
                            check_pred[num] = 1
                            check_true = 1
                            result_type1.append(true_text[0]) # 完全
                        else:
                            check_pred[num] = 1
                            check_true = 1
                            result_type2.append([true_text[0],pred_text[0]]) #一部一致
                            
                    if num == (len(pred_wh_dic[k])-1) and check_true == 0: # 正解データが、予想データの最後の要素まで一致しなかったとき
                        result_type3.append(true_text[0]) # 該当なし
                        
        miss_pred = [i for i,c in enumerate(check_pred) if c == 0]
        if len(miss_pred) > 0:
            for i in miss_pred:
                result_type4.append(pred_wh_dic[k][i][0]) # 誤予測
            
        report_v["完全"] = result_type1
        report_v["一部一致"] = result_type2
        report_v["該当なし"] = result_type3
        report_v["誤予測"] = result_type4            
        report_data[k] = report_v
        
    return report_data

In [9]:
# 4パターンの数を数える
def cal_report(file_id,result,wh_label):
    wh_num = [0 for i in range(5)]
    wh_num[0] = file_id
    for wh in wh_label:
        for k,v in result[wh].items():
            if k == "完全":
                wh_num[1] += len(v)
            elif k == "一部一致":
                wh_num[2] += len(v)
            elif k == "該当なし":
                wh_num[3] += len(v)
            elif k == "誤予測":
                wh_num[4] += len(v)
                    
    return wh_num  

In [16]:
"""
4タイプの結果の数を集計する
入力：ファイルid、4タイプの結果、ラベル
出力：1つのファイルにおける、ラベルごとのタイプごとの結果数
"""
def cal_report_wh(file_id,result):
    report_data= {}
    for label,wh_value in result.items():
        
        if label == "O": # ラベルOは除外する
            continue
            
        wh_num = [0 for i in range(5)]
        wh_num[0] = file_id
        for rtype,value in wh_value.items():
            if rtype == "完全":
                wh_num[1] += len(value)
            elif rtype == "一部一致":
                wh_num[2] += len(value)
            elif rtype == "該当なし":
                wh_num[3] += len(value)
            elif rtype == "誤予測":
                wh_num[4] += len(value)
            
        report_data[label] = wh_num
        
    return report_data

In [17]:
# テンプレートを作成する
def w2template(pred_wh_dic,text_list):
    
    template_txt = ""
    label_ids = []
    label_name = {}
    
    # 4W情報の保存
    for k,v in pred_wh_dic.items():
        if k == "O":
            continue
            
        if len(v) != 0:
            for v_i in v:
                # 要素の最初と最後
                sp_s = v_i[1][0]
                sp_e = v_i[1][-1] + 1
                
                # ラベル情報と要素
                label_name[sp_e] = k # ラベル情報保存
                label_ids.append(sp_s) # 要素の最初
                label_ids.append(sp_e) # 要素の最後
                      
    # テンプレート生成    
    data_text = text_list # 1つめのデータを利用

    for i,sp in enumerate(sorted(label_ids)):
        
        if i == 0: # 最初
            if sp != 0:
                template_txt += data_text[0:sp]
                
        elif i == (len(label_ids)-1): # 最後
            if sp < len(data_text):
                template_txt += "<{0}>".format(label_name[sp])
                template_txt += data_text[sp:len(data_text)]
                
        elif i%2 == 0:
            template_txt += data_text[start_i:sp]
        
        elif i%2 != 0:
            template_txt += "<{0}>".format(label_name[sp])
            start_i = sp  
            
    return template_txt


In [19]:
# ラベル評価 Leave one out

import csv

# ファイル書き出し用
ex_file = "r3_extraction_result.csv" # ルール3
f_file = "r3_failure_result.csv" # ルール3
ex_txt = "r3_failure_result.txt" # ルール3

ex_txt_file = open(ex_txt, 'w')  #書き込みモードでオープン
output_result = []
failure_result = []
wh_result = {"WHERE":[], "WHEN":[], "WHO":[], "WHAT":[], "HOW":[], "WHY":[],"SERIF":[]}

# ファイル読み込み用
c = CorpusReader('corpus_5w1hs.txt') # ファイル指定
all_sents = c.iob_sents('all') # データの読み込み
loo = LeaveOneOut() # LeaveOneOut呼び出し

for train_index, test_index in loo.split(all_sents):
    
    # 1. データ読み込み
    X_train = [sent2features(all_sents[i]) for i in train_index] # 学習データの特徴量
    y_train = [sent2labels(all_sents[i]) for i in train_index] # 学習データのラベル

    X_test = [sent2features(all_sents[i]) for i in test_index] # テストデータの特徴量
    y_test = [sent2labels(all_sents[i]) for i in test_index] # テストデータのラベル

    # 2.学習
    trainer = pycrfsuite.Trainer(verbose=False) # モデルの定義

    for xseq, yseq in zip(X_train, y_train):
        trainer.append(xseq, yseq) # 学習データの追加
    
    trainer.set_params({
        'c1': 1.0,   # coefficient for L1 penalty
        'c2': 1e-3,  # coefficient for L2 penalty
        'max_iterations': 50,  # stop earlier
        # include transitions that are possible, but not observed
        'feature.possible_transitions': True
    }) # パラメータの設定

    trainer.train('model.crfsuite') # モデル学習

    # 3.ラベル予測
    tagger = pycrfsuite.Tagger() #  pycrfsuiteのモデルを用意
    tagger.open('model.crfsuite') # モデルを開く
    tagger.info()
    
    
    # 4.ラベル予測結果
    # 4-1. データ用意
    test_id = test_index[0]
    example_sent = all_sents[test_id] 
    test_id += 1
    text_data = sent2tokens(example_sent) # テキスト
    text_line = "".join(text_data)
    
    c_data = sent2labels(example_sent) # 正解ラベルデータ
    p_data = tagger.tag(sent2features(example_sent)) # 予想ラベルデータ

    true_wh_dic = label_report(text_data,c_data) # 5w1hラベルごとに単語を保存する
    pred_wh_dic = label_report(text_data,p_data)  # 5w1hラベルごとに単語を保存する   
    result = eval_report(true_wh_dic,pred_wh_dic) # 結果を4パターンで出力
    wh_label = ["WHERE","WHO","WHEN","WHAT"] # ラベル定義
    c_r = cal_report(test_id,result,wh_label) # 4パターンの結果をまとめる
    output_result.append(c_r)
    
    # 結果表示
    print(test_id)
    ex_txt_file.writelines("==== {0} ====\n".format(test_id))
    ex_txt_file.writelines("{0}\n".format(text_line))
    ex_txt_file.writelines("\n")
    ex_txt_file.writelines("{0}\n".format(pred_wh_dic))
    ex_txt_file.writelines("{0}\n".format(w2template(pred_wh_dic,text_line)))
    
    ex_txt_file.writelines("-----------------------------\n")
    for wh_l in wh_label:
        ex_txt_file.writelines("==== {0} ====\n".format(wh_l))
        ex_txt_file.writelines("正解：{0}\n".format([t for t in true_wh_dic[wh_l]]))
        ex_txt_file.writelines("予想：{0}\n".format([t for t in pred_wh_dic[wh_l]]))
        ex_txt_file.writelines("該当なし：{0}\n".format(result[wh_l]["該当なし"]))
        ex_txt_file.writelines("誤予測：{0}\n".format(result[wh_l]["誤予測"]))
   
    ex_txt_file.writelines("-----------------------------\n") 
    ex_txt_file.writelines("{0}\n".format(["記事id","完全","一部一致","該当なし","誤予測"]))
    ex_txt_file.writelines("{0}\n".format(c_r))

    # 1記事における5w1hラベルごとの結果
    for k,v in cal_report_wh(test_id,result).items():
        wh_result[k].append(v) 
        
    # 失敗データのリスト化
    for wh_l in wh_label:
        for rt in result[wh_l]["該当なし"]:
            failure_result.append([test_id,wh_l,"該当なし",rt])
        for rt in result[wh_l]["誤予測"]:
            failure_result.append([test_id,wh_l,"誤予測",rt])            

ex_txt_file.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209


In [20]:
# ファイル書き込み
# 5W1Hラベルごとのデータ

for k,v in wh_result.items():
    wh_file = "r3_extraction_resul_{}.csv".format(k) # ルール3
    with open(wh_file, 'w') as f:
        writer = csv.writer(f, lineterminator='\n') # 改行コード（\n）を指定しておく
        writer.writerow(["記事id","完全","一部一致","該当なし","誤予測"])
        for o_data in v:
            writer.writerow(o_data)

In [21]:
# ファイル書き込み

# 評価結果
with open(ex_file, 'w') as f:
    writer = csv.writer(f, lineterminator='\n') # 改行コード（\n）を指定しておく
    writer.writerow(["記事id","完全","一部一致","該当なし","誤予測"])
    for o_data in output_result:
        writer.writerow(o_data)

# 失敗データ
with open(f_file, 'w') as f:
    writer = csv.writer(f, lineterminator='\n') # 改行コード（\n）を指定しておく
    writer.writerow(["記事id","ラベル","失敗タイプ","単語","原因"])
    for f_data in failure_result:
        writer.writerow(f_data)

In [ ]:
# サンプルソースコード
# 1.データ読み込み
#c = CorpusReader('corpus.txt') # ファイル指定
c = CorpusReader('corpus_5w1hs.txt') # ファイル指定
train_sents = c.iob_sents('train') # データの読み込み
test_sents = c.iob_sents('test') # データの読み込み

X_train = [sent2features(s) for s in train_sents] # 学習データの特徴量
y_train = [sent2labels(s) for s in train_sents] # 学習データのラベル

X_test = [sent2features(s) for s in test_sents] # テストデータの特徴量
y_test = [sent2labels(s) for s in test_sents] # テストデータのラベル

# 2.学習
trainer = pycrfsuite.Trainer(verbose=False) # モデルの定義

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq) # 学習データの追加
    
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
}) # パラメータの設定

trainer.train('model.crfsuite') # モデル学習

# 3.ラベル予測・評価
tagger = pycrfsuite.Tagger() #  pycrfsuiteのモデルを用意
tagger.open('model.crfsuite') # モデルを開く
tagger.info()

# 4.テストの用意
example_sent = test_sents[0]
text_data = sent2tokens(example_sent)
p_data = tagger.tag(sent2features(example_sent))
c_data = sent2labels(example_sent)

wh_label = ["WHERE","WHO","WHEN","WHAT"]
true_wh_dic = label_report(text_data,c_data)
pred_wh_dic = label_report(text_data,p_data)

result = eval_report(true_wh_dic,pred_wh_dic)
c_r = cal_report(file_id,result,wh_label)


# 結果表示
text_line = "".join(text)
print(text_line)
print(" ")
print(w2template(pred_wh_dic,text_line))    
    
print("-----------------------------")    
for wh_l in wh_label:
    print("==== {0} ====".format(wh_l))
    print("正解：{0}".format([t for t in true_wh_dic[wh_l]]))
    print("予想：{0}".format([t for t in pred_wh_dic[wh_l]]))
    print("該当なし：{0}".format(result[wh_l]["該当なし"]))
    print("誤予測：{0}".format(result[wh_l]["誤予測"]))

print("-----------------------------") 
print(["記事id","完全","一部一致","該当なし","誤予測"])
print(c_r)    

#print("----------------------")
#y_pred = [tagger.tag(xseq) for xseq in X_test] # テストデータにタグ付け
#print(bio_classification_report(y_test, y_pred))